In [27]:
import datetime as dt
import pandas as pd
import numpy as np

In [28]:
excel_file = '/home/siddharth/Desktop/FT/CodeChallenge/vividmoney/Customers Monthly.csv'
customer_monthly = pd.read_csv(excel_file)

In [29]:
customer_monthly['CUSTOMER_HK'].describe()  

count                                                 39535
unique                                                19717
top       238fe4ef9aa063a4ffe2424a20ead5ab9d269cf43ac328...
freq                                                     69
Name: CUSTOMER_HK, dtype: object

In [26]:

customer_monthly.drop_duplicates(keep = 'first',inplace = True)

In [30]:
customer_monthly.DAY_DT = pd.to_datetime(customer_monthly.DAY_DT)
customer_monthly.dtypes

CUSTOMER_HK                 object
DAY_DT              datetime64[ns]
BANK_IS_ACTIVE                bool
INVEST_IS_ACTIVE              bool
PLAN_NM                     object
dtype: object

In [31]:
def get_day(x): 
    return dt.datetime(x.year, x.month, x.day) 

In [32]:
# Create InvoiceDay column
customer_monthly['InvoiceDay_DT'] = customer_monthly['DAY_DT'].apply(get_day) 

# Group by CustomerID and select the InvoiceDay value
grouping = customer_monthly.groupby('CUSTOMER_HK')['InvoiceDay_DT'] 

# Assign a minimum InvoiceDay value to the dataset
customer_monthly['CohortDay'] = grouping.transform('min')

# View the top 5 rows
customer_monthly.head()

,CUSTOMER_HK,DAY_DT,BANK_IS_ACTIVE,INVEST_IS_ACTIVE,PLAN_NM,InvoiceDay_DT,CohortDay
0,04c604e678b18d7ca10b85fb8d3672a03b7b6b71e78dd5...,2020-11-30,True,False,prime,2020-11-30,2020-11-30
1,a42a9fdf0ee31d331c9d96ce6fbf30f307b52ff40a55ec...,2021-01-31,True,False,prime,2021-01-31,2020-11-30
2,be92b2698f17aa7fab796fd5eb132cc251fce6fe1870dc...,2020-12-31,True,False,prime,2020-12-31,2020-12-31
3,c84455bb3bbc2dbf16a1edd9c44e76baaf06b0ee224ed5...,2020-12-31,True,False,prime,2020-12-31,2020-12-31
4,7df3aa4e5f29e4f5946d652d4f6d6f88d3f850ac9ba33a...,2020-12-31,False,False,prime,2020-12-31,2020-12-31


In [33]:
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

In [34]:
# Get the integers for date parts from the InvoiceDaycolumn
invoice_year, invoice_month, invoice_day = get_date_int(customer_monthly, 'InvoiceDay_DT')

# Get the integers for date parts from the CohortDay column
cohort_year, cohort_month, cohort_day = get_date_int(customer_monthly, 'CohortDay')

In [35]:
# Calculate difference in years
years_diff = invoice_year - cohort_year

# Calculate difference in months
months_diff = invoice_month - cohort_month

# Calculate difference in days
days_diff = invoice_day - cohort_day

# Extract the difference in days from all previous values
customer_monthly['CohortIndex'] = years_diff * 365 + months_diff * 30 + days_diff + 1

customer_monthly.head()

,CUSTOMER_HK,DAY_DT,BANK_IS_ACTIVE,INVEST_IS_ACTIVE,PLAN_NM,InvoiceDay_DT,CohortDay,CohortIndex
0,04c604e678b18d7ca10b85fb8d3672a03b7b6b71e78dd5...,2020-11-30,True,False,prime,2020-11-30,2020-11-30,1
1,a42a9fdf0ee31d331c9d96ce6fbf30f307b52ff40a55ec...,2021-01-31,True,False,prime,2021-01-31,2020-11-30,67
2,be92b2698f17aa7fab796fd5eb132cc251fce6fe1870dc...,2020-12-31,True,False,prime,2020-12-31,2020-12-31,1
3,c84455bb3bbc2dbf16a1edd9c44e76baaf06b0ee224ed5...,2020-12-31,True,False,prime,2020-12-31,2020-12-31,1
4,7df3aa4e5f29e4f5946d652d4f6d6f88d3f850ac9ba33a...,2020-12-31,False,False,prime,2020-12-31,2020-12-31,1


In [36]:
# Define a function that will parse the month from the date 
def get_month(x): 
    return dt.datetime(x.year, x.month, 1)

In [40]:
# Extract the difference in months from all previous values
customer_monthly['CohortIndex_M'] = years_diff * 12 + months_diff + 1

# Create InvoiceMonth column
customer_monthly['InvoiceMonth'] = customer_monthly['DAY_DT'].apply(get_month)

# Group by CustomerID and select the InvoiceMonth value
grouping = customer_monthly.groupby('CUSTOMER_HK')['InvoiceMonth']

# Assign a minimum InvoiceMonth value to the dataset
customer_monthly['CohortMonth'] = grouping.transform('min')

# View the top 5 rows
customer_monthly

,CUSTOMER_HK,DAY_DT,BANK_IS_ACTIVE,INVEST_IS_ACTIVE,PLAN_NM,InvoiceDay_DT,CohortDay,CohortIndex,CohortIndex_M,InvoiceMonth,CohortMonth
0,04c604e678b18d7ca10b85fb8d3672a03b7b6b71e78dd5...,2020-11-30,True,False,prime,2020-11-30,2020-11-30,1,1,2020-11-01,2020-11-01
1,a42a9fdf0ee31d331c9d96ce6fbf30f307b52ff40a55ec...,2021-01-31,True,False,prime,2021-01-31,2020-11-30,67,3,2021-01-01,2020-11-01
2,be92b2698f17aa7fab796fd5eb132cc251fce6fe1870dc...,2020-12-31,True,False,prime,2020-12-31,2020-12-31,1,1,2020-12-01,2020-12-01
3,c84455bb3bbc2dbf16a1edd9c44e76baaf06b0ee224ed5...,2020-12-31,True,False,prime,2020-12-31,2020-12-31,1,1,2020-12-01,2020-12-01
4,7df3aa4e5f29e4f5946d652d4f6d6f88d3f850ac9ba33a...,2020-12-31,False,False,prime,2020-12-31,2020-12-31,1,1,2020-12-01,2020-12-01
...,...,...,...,...,...,...,...,...,...,...,...
39530,4c65aeb4e3c56d9ad692a2dd53f6e58d3b1c03f53ebee8...,2021-01-31,True,False,prime,2021-01-31,2020-12-31,36,2,2021-01-01,2020-12-01
39531,b3c4de44a98d55acc296d54636f321323e55bc793f0c01...,2020-12-31,True,False,prime,2020-12-31,2020-12-31,1,1,2020-12-01,2020-12-01
39532,ed1cd862b1e0c82f9c96b6a895c6122fbfcac469801701...,2020-12-31,True,False,prime,2020-12-31,2020-11-30,32,2,2020-12-01,2020-11-01
39533,68b47f0f4243f0fccc0e21bc72b750f76387838e20aa82...,2020-11-30,True,False,prime,2020-11-30,2020-11-30,1,1,2020-11-01,2020-11-01


In [39]:
# Group by CohortMonth and CohortIndex
grouping = customer_monthly.groupby(['CohortMonth', 'CohortIndex_M'])

# Count the number of customers in each group
cohort_data = grouping['CUSTOMER_HK'].apply(pd.Series.nunique)

# Reset the index
cohort_data = cohort_data.reset_index()

# Create a pivot table with CohortMonth as rows, CohortIndex as the columns, and CustomerID counts as the values
cohort_counts = cohort_data.pivot(index='CohortMonth', columns='CohortIndex_M', values='CUSTOMER_HK')

cohort_counts

CohortIndex_M,1,2,3
CohortMonth,,,
2020-11-01,5808.0,2374.0,2338.0
2020-12-01,6298.0,2337.0,NaN
2021-01-01,7611.0,NaN,NaN


In [42]:
# Counting daily active user from each chort
grouping = customer_monthly.groupby(['CohortMonth', 'CohortIndex'])
# Counting number of unique customer Id's falling in each group of CohortMonth and CohortIndex
cohort_data = grouping['CUSTOMER_HK'].apply(pd.Series.nunique)
cohort_data = cohort_data.reset_index()
 # Assigning column names to the dataframe created above
cohort_counts = cohort_data.pivot(index='CohortMonth',
                                 columns ='CohortIndex',
                                 values = 'CUSTOMER_HK')
# Printing top 5 rows of Dataframe
cohort_data.head()

,CohortMonth,CohortIndex,CUSTOMER_HK
0,2020-11-01,1,5808
1,2020-11-01,32,2374
2,2020-11-01,67,2338
3,2020-12-01,1,6298
4,2020-12-01,36,2337


In [43]:
cohort_sizes = cohort_counts.iloc[:,0]
retention = cohort_counts.divide(cohort_sizes, axis=0)
# Coverting the retention rate into percentage and Rounding off.
retention.round(3)*100

CohortIndex,1,32,36,67
CohortMonth,,,,
2020-11-01,100.0,40.9,NaN,40.3
2020-12-01,100.0,NaN,37.1,NaN
2021-01-01,100.0,NaN,NaN,NaN
